# Pruebas automatizadas datos humedad relativa - Clase

> Elaborado por Paola Álvarez, profesional contratista IDEAM, contrato 196 de 2024. Comentarios o inquietudes, remitir a *palvarez@ideam.gov.co* 

**Librerías**

In [1]:
import pandas as pd
import numpy as np
import os
import re
import logging
from functools import wraps

----

A continuación, se encuentran las pruebas de pre-validación de datos de EMA para verificar su capacidad de detección de datos

## Clase con métodos de aplicación de QC

In [2]:
# Configuración del logger para guardar en el directorio de archivos y sobrescribir cada vez
def setup_logger(log_file_path):
    logger = logging.getLogger('Test_QC')
    logger.setLevel(logging.INFO)
    # Clear existing handlers to avoid duplicate logs
    if logger.hasHandlers():
        logger.handlers.clear()
    file_handler = logging.FileHandler(log_file_path, mode='a')
    file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
    logger.addHandler(file_handler)
    return logger

def log_failures(func):
    @wraps(func)
    def wrapper(self, chunk, archivo):
        try:
            result, mask = func(self, chunk, archivo)
            if mask is not None:
                try:
                    aligned_mask = mask.reindex(chunk.index, fill_value=False)  # Asegura que la máscara esté alineada con el índice del DataFrame
                except AttributeError:
                    aligned_mask = mask  # Si no se puede reindexar, usa la máscara tal como está
                for index, row in chunk[aligned_mask].iterrows():
                    self.logger.info('Archivo: %s - Fila: %s - Valor fallido en %s: %s', archivo, index, func.__name__, row['Valor'])
            return result
        except ValueError as e:
            self.logger.error('Error procesando el archivo %s: %s', archivo, str(e))
            return chunk  # Devuelve una máscara falsa para manejar el error
    return wrapper

In [4]:
class AutomatHREMA:
    
    def __init__(self, dir_files, chunk_size=54000):
        self.dir_files = dir_files
        self.ruta_archivos = os.listdir(dir_files)
        self.chunk_size = chunk_size
        self.last_rows = None
        self.current_file = None
        # Sección configuración de logs
        log_file_path = os.path.join(dir_files, 'QC_HR.log')
        self.logger = setup_logger(log_file_path)
        self.logger.info('Inicialización de PreValidPatmEMA en directorio: %s', dir_files)

    def p_transm(self, chunk, archivo):
        '''Esta prueba verifica si existe al menos el 70% de datos esperados por día y hora
        en la serie de datos; aquellos que no superen la prueba, son marcados como sospechosos'''
        # Se encontraron diferentes frecuencias en la transmisión de Patm, por lo tanto:
        freqinst200b = pd.read_csv('EMAHR_Allinfo_Replcbl.csv', encoding='latin-1') #, sep=';')
        
        # Se define un diccionario de frecuencias y cantidades esperadas
        frecuencias = {
            'T': {'cant_esperd_h': 60, 'cant_esperd_d': 1440},
            '5T': {'cant_esperd_h': 12, 'cant_esperd_d': 288},
            '10T': {'cant_esperd_h': 6, 'cant_esperd_d': 144},
            'H': {'cant_esperd_h': 1, 'cant_esperd_d': 24}
        }
        
        # Obtener la frecuencia de 'freqinst200b' basado en 'Station' y asignar a 'periodos'
        station_value = chunk['Station'].values[0]
        if pd.isna(station_value):
            periodos = None
        else:
            freqinst200b_station = freqinst200b.loc[freqinst200b['Station'] == station_value]
            if freqinst200b_station.empty:
                print(f"No se encontró la estación {station_value} en freqinst200b")
                return chunk
    
            freq_inf_value = freqinst200b_station['FreqInf'].values[0]
    
            if pd.isna(freq_inf_value):
                try:
                    periodos = pd.infer_freq(chunk['Fecha'][-25:])
                    print(periodos)
                    if periodos is None:
                        print(f"Frecuencia inferida es None para el archivo {archivo}")
                        return chunk
                except ValueError as e:
                    print(f'Error al inferir la frecuencia en el archivo {archivo}: {str(e)}')
                    return chunk
            else:
                periodos = freq_inf_value
    
        if periodos is None:
            print(f"Periodo es None para el archivo {archivo}")
            return chunk
    
        # Obtener las cantidades esperadas de acuerdo a la frecuencia
        cant_esperd_h = frecuencias[periodos]['cant_esperd_h']
        cant_esperd_d = frecuencias[periodos]['cant_esperd_d']
    
        # Se establecen los aceptables
        cant_aceptab_hora = 0.7 * cant_esperd_h
        cant_aceptab_dia = 0.7 * cant_esperd_d
    
        # Agregar columna de etiquetas al dataframe original
        chunk['Estado'] = np.nan
        
        # Definir función para asignar etiquetas
        def asignar_etiqueta(row):
            if row['count'] < cant_aceptab_hora:
                chunk.loc[chunk['Fecha'].dt.floor('H') == row['Fecha'].floor('H'), 'Estado'] = '0PSO0'
        
        # Evaluar por cada grupo de datos por hora y asignar la etiqueta
        canthora = chunk.groupby(chunk['Fecha'].dt.floor('H')).size().reset_index(name='count')
        canthora.apply(asignar_etiqueta, axis=1)
        
        # Definir función para asignar etiquetas de acumulado diario
        def asignar_etiqueta_diaria(row):
            if row['count'] < cant_aceptab_dia:
                chunk.loc[chunk['Fecha'].dt.floor('D') == row['Fecha'].floor('D'), 'Estado'] = '0PSO0'
        
        # Evaluar por cada grupo de datos por día y asignar la etiqueta
        cantdia = chunk.groupby(chunk['Fecha'].dt.floor('D')).size().reset_index(name='count')
        cantdia.apply(asignar_etiqueta_diaria, axis=1)
        
        return chunk, _
        
    @log_failures
    def p_estruct(self, chunk, archivo):
        '''Esta prueba verifica si los datos fueron transmitidos en horas y minutos exactos al ser el
        comportamiento esperado'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = None

        freqinst200b = pd.read_csv('EMAHR_Allinfo_Replcbl.csv', encoding='latin-1') #, sep=';')
        
        # Define un diccionario de frecuencias y cantidades esperadas
        frecuencias = {
            '5T': {'num_para_modulo': 5},
            '10T': {'num_para_modulo': 10}
        }
        
        # Obtener la frecuencia de 'freqinst200b' basado en 'Station' y asignar a 'periodos'
        station_value = chunk['Station'].values[0]
        if pd.isna(station_value):
            periodos = None
        else:
            freqinst200b_station = freqinst200b.loc[freqinst200b['Station'] == station_value]
            if freqinst200b_station.empty:
                print(f"No se encontró la estación {station_value} en freqinst200b")
                return chunk
            freq_inf_value = freqinst200b_station['FreqInf'].values[0]
    
            if pd.isna(freq_inf_value):
                try:
                    periodos = pd.infer_freq(chunk['Fecha'][-25:])
                    print(periodos)
                    if periodos is None:
                        print(f"Frecuencia inferida es None para el archivo {archivo}")
                        return chunk
                except ValueError as e:
                    print(f'Error al inferir la frecuencia en el archivo {archivo}: {str(e)}')
                    return chunk
            else:
                periodos = freq_inf_value
    
        # Se hace frente al caso de no encontrar la estación
        if periodos is not None:
            
            # Generar la operación para observar si la estructura es exacta en minutos
            fecha = chunk['Fecha']
    
            # Se vectoriza la evaluación de la estructura por minuto
            # Para cada chunk:
            if periodos == 'T':
                mask = fecha.dt.second != 0
            elif periodos == 'H':
                mask = (fecha.dt.minute != 0) | (fecha.dt.second != 0)
            else:
                # Se obtiene num_para_modulo
                num_para_modulo = frecuencias[periodos]['num_para_modulo']
                mask = fecha.dt.minute % num_para_modulo != 0
    
            # Cambiar '0PSO0' a '0PSO1' donde la máscara se cumple
            chunk.loc[mask & (chunk['Estado'] == '0PSO0'), 'Estado'] = '0PSO1'
            # Asignar '0PSO0' a los NaN donde la máscara se cumple
            chunk.loc[mask & chunk['Estado'].isnull(), 'Estado'] = '0PSO0'
        
        else:  # Si el periodo es None, no se hace ninguna modificación al chunk, pero puedes imprimir un mensaje si quieres
            print(f"No se encontró la frec de la estac.{station_value} ni un proyecto respectivo en freqinst200b")
            
        return chunk, mask

    @log_failures
    def p_limrig(self, chunk, archivo):
        '''Esta prueba verifica si los datos crudos se encuentran fuera del umbral físico inferior o superior'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = None
            
        # Se genera la columna de estado anterior
        chunk['Estado_Anterior'] = np.nan
        
        # Se establecen los umbrales físicos/rígidos a datos crudos en nuevas colummnas para vectorizar
        chunk['umbr_crud_inf'] = 0.0
        chunk['umbr_crud_sup'] = 100.0

        # Compara el dato con umbrales inferiores y superiores 
        mask_outbounds = (chunk['Valor'] < chunk['umbr_crud_inf']) | (chunk['Valor'] > chunk['umbr_crud_sup'])

        # Lógica de etiquetado para 'Estado_Anterior'
        condicion_0PSO0 = mask_outbounds & chunk['Estado'].isin(['0PSO0', '0PSO1'])
        chunk.loc[condicion_0PSO0, 'Estado_Anterior'] = chunk.loc[condicion_0PSO0, 'Estado']

        # Lógica de etiquetado para 'Estado'
        condicion_0PER0 = mask_outbounds & (chunk['Estado'].isnull() | chunk['Estado'].isin(['0PSO0', '0PSO1']))
        chunk.loc[condicion_0PER0, 'Estado'] = '0PER0'
        
        # Se eliminan las columnas no deseadas
        if 'umbr_crud_inf' in chunk.columns:
            chunk.drop(columns=['umbr_crud_inf', 'umbr_crud_sup'], axis=1, inplace=True)
                
        return chunk, mask_outbounds

    @log_failures
    def p_persist(self, chunk, archivo):
        '''Esta prueba detecta los datos que se repiten por más de cuatro horas consecutivas'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = None
                
        # Se genera la columna 'Estado_anterior' si no existe
        if 'Estado_Anterior' not in chunk.columns:
            chunk['Estado_Anterior'] = None

        # Verificar si el archivo ha cambiado
        if self.current_file != archivo:
            # Si el archivo cambió, resetea self.last_rows y actualiza self.current_file
            self.last_rows = None
            self.current_file = archivo
            
        # Usar self.last_rows para concatenar con el chunk actual
        if self.last_rows is not None:
            chunk = pd.concat([self.last_rows, chunk])
            chunk.reset_index(drop=True)

        # Se crean máscaras para el intervalo del día con radiación solar que puede afectar la humedad
        mask_sunny = (chunk['Fecha'].dt.hour >= 4) & (chunk['Fecha'].dt.hour <= 20)
        # Se filtran los datos para esas horas
        mask_sun = chunk[mask_sunny]
            
        # Crear máscaras para cada comparación de las 4 filas consecutivas
        mask_1 = (mask_sun['Valor'] == mask_sun['Valor'].shift(1))
        mask_2 = (mask_sun['Valor'] == mask_sun['Valor'].shift(2))
        mask_3 = (mask_sun['Valor'] == mask_sun['Valor'].shift(3))
        mask_4 = (mask_sun['Valor'] == mask_sun['Valor'].shift(4))
        
        # Combinar todas las máscaras para obtener la condición deseada
        mask_pers4datos = mask_1 & mask_2 & mask_3 & mask_4
        
        # Etiquetado de valores, se inicia con el Estado Anterior
        condicion_0PSO0 = mask_pers4datos & chunk['Estado'].isin(['0PSO0', '0PSO1'])
        chunk.loc[condicion_0PSO0, 'Estado_Anterior'] = chunk.loc[condicion_0PSO0, 'Estado']

        # Lógica de etiquetado para 'Estado'
        condicion_0PER0 = mask_pers4datos & (chunk['Estado'].isnull() | chunk['Estado'].isin(['0PSO0', '0PSO1']))
        chunk.loc[condicion_0PER0, 'Estado'] = '0PER0'
        mask_pers4datos = mask_pers4datos & ~condicion_0PER0

        condicion_0PER1 = mask_pers4datos & (chunk['Estado'] == '0PER0')
        chunk.loc[condicion_0PER1, 'Estado'] = '0PER1'

        self.last_rows = chunk.tail(4)
        
        return chunk, mask_pers4datos

    def p_salto(self, chunk, archivo):
        '''Esta prueba verifica si la variación entre valores consecutivos excede 45.0 %'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = None
        # Se genera la columna 'Estado_anterior' si no existe
        if 'Estado_Anterior' not in chunk.columns:
            chunk['Estado_Anterior'] = None
            
        # Se toma nuevamente el archivo de frecuencias para analizar datos estrictamente consecutivos
        freqinst200b = pd.read_csv('EMAHR_Allinfo_Replcbl.csv', encoding='latin-1') #, sep=';')

        # Obtener la frecuencia de 'freqinst200b' basado en 'Station' y asignar a 'periodos'
        sttn_code = chunk['Station'].values[0]
        if pd.isna(sttn_code):
            periodos = None
        else:
            freqinst200b_station = freqinst200b.loc[freqinst200b['Station'] == sttn_code]
            if freqinst200b_station.empty:
                print(f"No se encontró la estación {sttn_code} en freqinst200b")
                return chunk
            # Se toma el valor inferido de la frecuencia
            freq_inf_value = freqinst200b_station['FreqInf'].values[0]
            # Si el valor inferido es un NaN, se infiere dentro del código (esto porque hay estaciones con )
            if pd.isna(freq_inf_value):
                try:
                    periodos = pd.infer_freq(chunk['Fecha'][-25:])
                    print(periodos)
                    if periodos is None:
                        print(f"Frecuencia inferida es None para el archivo {archivo}")
                        return chunk
                except ValueError as e:
                    print(f'Error al inferir la frecuencia en el archivo {archivo}: {str(e)}')
                    return chunk
            else:
                periodos = freq_inf_value
    
        if periodos is None:
            print(f"Periodo es None para el archivo {archivo}")
            return chunk

        # Asegurarse de que 'periodos' tenga un número antes de la unidad
        if periodos.isalpha():
            periodos = '1' + periodos
        
        # Ordenar el chunk por la columna 'Fecha'
        chunk = chunk.sort_values('Fecha').reset_index(drop=True)

        # Crear una columna de diferencia temporal
        chunk['Fecha_anterior'] = chunk['Fecha'].shift(1)
        chunk['Delta_tiempo'] = chunk['Fecha'] - chunk['Fecha_anterior']
        
        # Crear una máscara para identificar filas consecutivas según la frecuencia esperada
        mask_consecutivo = chunk['Delta_tiempo'] == pd.to_timedelta(periodos)
        
        # Calcular la diferencia absoluta entre los valores consecutivos
        chunk['Delta'] = chunk['Valor'].diff().abs()
        chunk['Delta'] = chunk['Delta'].where(mask_consecutivo)

        # Máscara para identificar variaciones mayores a 45.0
        mask_variacion = chunk['Delta'] > 45.0
      
        # Etiquetado de valores, se inicia con el Estado Anterior
        condicion_0PSO0 = mask_variacion & chunk['Estado'].isin(['0PSO0', '0PSO1'])
        chunk.loc[condicion_0PSO0, 'Estado_Anterior'] = chunk.loc[condicion_0PSO0, 'Estado']

        # Lógica de etiquetado para 'Estado' - '0PER0'
        condicion_0PER0 = mask_variacion & (chunk['Estado'].isnull() | chunk['Estado'].isin(['0PSO0', '0PSO1']))
        chunk.loc[condicion_0PER0, 'Estado'] = '0PER0'
        mask_variacion = mask_variacion & ~condicion_0PER0
        # '0PER1'
        condicion_0PER1 = mask_variacion & (chunk['Estado'] == '0PER0')
        chunk.loc[condicion_0PER1, 'Estado'] = '0PER1'
        mask_variacion = mask_variacion & ~condicion_0PER1
        # '0PER2'
        condicion_0PER2 = mask_variacion & (chunk['Estado'] == '0PER1')
        chunk.loc[condicion_0PER2, 'Estado'] = '0PER2'

        # Eliminar las columnas temporales antes de devolver el chunk
        chunk.drop(columns=['Delta', 'Fecha_anterior', 'Delta_tiempo'], axis=1, inplace=True)

        return chunk, mask_variacion

    def p_horavmaxmin(self, chunk, archivo):
        '''Esta prueba detecta los datos que son máximos y mínimos en horarios distintos a los posibles por la temperatura máxima en el
        día según la radiación solar'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = None
        # Se genera la columna 'Estado_anterior' si no existe
        if 'Estado_Anterior' not in chunk.columns:
            chunk['Estado_Anterior'] = None

        if chunk['Estado'].notna().all(): # Se verifica que no hayan valores nulos en tal columna
            chunk_hmm = chunk[~chunk['Estado'].str.startswith('0PER', na=False)].copy()
        else:
            # Si todos los valores son NaN, simplemente copia el chunk
            chunk_hmm = chunk.copy()
            
        # Se crean máscaras para los intervalos de tiempo conocidos para valores máximos y mínimos
        mask_max_morning = (chunk_hmm['Fecha'].dt.hour >= 0) & (chunk_hmm['Fecha'].dt.hour < 10) 
        mask_max_afternoon = (chunk_hmm['Fecha'].dt.hour > 17) & (chunk_hmm['Fecha'].dt.hour <= 23)
        mask_min_afternoon = (chunk_hmm['Fecha'].dt.hour >= 10) & (chunk_hmm['Fecha'].dt.hour <= 17)
        
        # Se filtran los datos para esas horas
        max_validdata = chunk_hmm[mask_max_morning | mask_max_afternoon]
        min_validdata = chunk_hmm[mask_min_afternoon]

        # Encontrar dos valores máximos por día
        max_values = chunk_hmm.groupby(chunk_hmm['Fecha'].dt.date).apply(lambda x: x.nlargest(2, 'Valor')).reset_index(level=0, drop=True)
        
        # Encontrar dos valores mínimos por día
        min_values = chunk_hmm.groupby(chunk_hmm['Fecha'].dt.date).apply(lambda x: x.nsmallest(2, 'Valor')).reset_index(level=0, drop=True)

        # Verificar los máximos y mínimos y obtener las horas correspondientes
        notvalid_max_values = max_values[~max_values.index.isin(max_validdata.index)]
        notvalid_min_values = min_values[~min_values.index.isin(min_validdata.index)]

        # Combinar los valores no válidos en un solo DataFrame
        notvalid_values = pd.concat([notvalid_max_values, notvalid_min_values])
        # Crear una máscara para identificar los índices de los valores no válidos
        notval_maxmin = chunk_hmm.index.isin(notvalid_values.index)

        ## Actualización de estado
        # Condición llenado de 'Estado_Anterior', si aplica
        condicion_0PSO0 = notval_maxmin & chunk_hmm['Estado'].isin(['0PSO0', '0PSO1'])
        chunk_hmm.loc[condicion_0PSO0, 'Estado_Anterior'] = chunk_hmm.loc[condicion_0PSO0, 'Estado']

        # Se etiquetan los atípicos
        condicion_0PAT0 = notval_maxmin & (chunk_hmm['Estado'].isnull() | chunk_hmm['Estado'].isin(['0PSO0', '0PSO1']))
        chunk_hmm.loc[condicion_0PAT0, 'Estado'] = '0PAT0'

        # Se copia al chunk original
        chunk.loc[chunk_hmm.index] = chunk_hmm
        return chunk, notval_maxmin

    def p_sigma(self, chunk, archivo):
        '''Esta prueba calcula, con los datos no etiquetados en la anterior prueba, la 4sigmas +- la media para detectar
        datos atípicos en los conjuntos de los datos'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = None
        # Se genera la columna 'Estado_anterior' si no existe
        if 'Estado_Anterior' not in chunk.columns:
            chunk['Estado_Anterior'] = None

        if chunk['Estado'].notna().all(): # Se verifica que no hayan valores nulos en tal columna
            chunk_sgm = chunk[~chunk['Estado'].str.startswith('0PER', na=False)].copy()
        else:
            # Si todos los valores son NaN, simplemente copia el chunk
            chunk_sgm = chunk.copy()

        # Se calculan los estadísticos para sigma
        mean = chunk_sgm['Valor'].mean()
        std = chunk_sgm['Valor'].std()
        # Con ellos, se establecen los límites superior e inferior
        chunk_sgm['LimSup_Sigma'] = (mean + (4 * std))
        chunk_sgm['LimInf_Sigma'] = (mean - (4 * std))

        # Se etiquetan los valores que sobrepasen el límite
        mask_outbsigma = (chunk_sgm['Valor'] < chunk_sgm['LimInf_Sigma']) | (chunk_sgm['Valor'] > chunk_sgm['LimSup_Sigma'])
        
        ## Actualización de estado
        # Condición llenado de 'Estado_Anterior', si aplica
        condicion_0PSO0 = mask_outbsigma & chunk_sgm['Estado'].isin(['0PSO0', '0PSO1'])
        chunk_sgm.loc[condicion_0PSO0, 'Estado_Anterior'] = chunk_sgm.loc[condicion_0PSO0, 'Estado']

        # Se etiquetan los atípicos - '0PAT0'
        condicion_0PAT0 = mask_outbsigma & (chunk_sgm['Estado'].isnull() | chunk_sgm['Estado'].isin(['0PSO0', '0PSO1']))
        chunk_sgm.loc[condicion_0PAT0, 'Estado'] = '0PAT0'
        mask_outbsigma = mask_outbsigma & ~condicion_0PAT0
        # 0PAT1
        condicion_0PAT1 = mask_outbsigma & (chunk_sgm['Estado'] == '0PAT0')
        chunk_sgm.loc[condicion_0PAT1, 'Estado'] = '0PAT1'
        # mask_outbsigma = mask_outbsigma & ~condicion_0PAT1
        # # 0PAT2
        # condicion_0PAT2 = mask_outbsigma & (chunk_sgm['Estado'] == '0PAT1')
        # chunk_sgm.loc[condicion_0PAT2, 'Estado'] = '0PAT2'
                    
        # Se eliminan las columnas no deseadas
        if 'LimSup_Sigma' in chunk.columns:
            chunk.drop(columns=['LimSup_Sigma', 'LimInf_Sigma'], axis=1, inplace=True)
    
        chunk.loc[chunk_sgm.index] = chunk_sgm
        return chunk, mask_outbsigma

    @log_failures
    def p_coherPFvals(self, chunk, archivo):
        '''Esta prueba verifica que un valor tenga coherencia con los 5 anteriores según su desviación estándar y media'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = None    
        # Se genera la columna 'Estado_anterior' si no existe
        if 'Estado_Anterior' not in chunk.columns:
            chunk['Estado_Anterior'] = None
            
        ## Trabajo con frecuencias
        # Se toma nuevamente el archivo de frecuencias para analizar datos estrictamente consecutivos
        freqinst200b = pd.read_csv('EMAHR_Allinfo_Replcbl.csv', encoding='latin-1')
        # Obtener la frecuencia de 'freqinst200b' basado en 'Station' y asignar a 'periodos'
        sttn_code = chunk['Station'].values[0]
        if pd.isna(sttn_code):
            periodos = None
        else:
            freqinst200b_station = freqinst200b.loc[freqinst200b['Station'] == sttn_code]
            if freqinst200b_station.empty:
                print(f"No se encontró la estación {sttn_code} en freqinst200b")
                return chunk, None
            # Se toma el valor inferido de la frecuencia
            freq_inf_value = freqinst200b_station['FreqInf'].values[0]
            # Si el valor inferido es un NaN, se infiere dentro del código
            if pd.isna(freq_inf_value):
                try:
                    periodos = pd.infer_freq(chunk['Fecha'][-25:])
                    if periodos is None:
                        print(f"Frecuencia inferida es None para el archivo {archivo}")
                        return chunk, None
                except ValueError as e:
                    print(f'Error al inferir la frecuencia en el archivo {archivo}: {str(e)}')
                    return chunk, None
            else:
                periodos = freq_inf_value
    
        if periodos is None:
            print(f"Periodo es None para el archivo {archivo}")
            return chunk, None

        ## Se asegura la revisión de 5 datos anteriores aún si cambia el chunk
        # Verificar si el archivo ha cambiado
        if self.current_file != archivo:
            # Si el archivo cambió, resetea self.last_rows y actualiza self.current_file
            self.last_rows = None
            self.current_file = archivo
        # Usar self.last_rows para concatenar con el chunk actual
        if self.last_rows is not None:
            chunk = pd.concat([self.last_rows, chunk])
            chunk.reset_index(drop=True)
    
        # Ordenar el chunk por la columna 'Fecha'
        chunk = chunk.sort_values('Fecha').reset_index(drop=True)

        # Asegurarse de que 'periodos' tenga un número antes de la unidad
        if periodos.isalpha():
            periodos = '1' + periodos

        # Crear una columna de diferencia temporal
        chunk['Fecha_anterior'] = chunk['Fecha'].shift(1)
        chunk['Delta_tiempo'] = chunk['Fecha'] - chunk['Fecha_anterior']

        # Se genera filtro para no considerar datos ya catalogados como erróneos
        if 'Estado' not in chunk.columns or chunk['Estado'].isnull().all():
            chunk_PFvals = chunk.copy()
        else:
            chunk_PFvals = chunk[~chunk['Estado'].str.startswith(('0PER','0PAT'), na=False)].copy()

        # Crear una máscara para identificar filas consecutivas según la frecuencia esperada
        mask_consecutivo = chunk_PFvals['Delta_tiempo'] == pd.to_timedelta(periodos)
        chunk_PFvals['consec_group'] = (~mask_consecutivo).cumsum()

        # Se establecen diferentes ventanas según frecuencias
        windows = {'T': {'window': 240}, '5T': {'window': 72}, '10T': {'window': 48}, 'H': {'window': 11}}
        window_size = windows[periodos]['window']
        half_window = window_size // 2

        # Filtrar grupos que tienen al menos el tamaño de ventana necesario
        group_counts = chunk_PFvals['consec_group'].value_counts()
        valid_groups = group_counts[group_counts >= window_size].index
        chnk_cohPFvl = chunk_PFvals[chunk_PFvals['consec_group'].isin(valid_groups)]

        # Verificar que los datos anteriores y posteriores sean consecutivos
        valid_indices = []
        for i in range(half_window, len(chnk_cohPFvl) - half_window):
            if all(mask_consecutivo[i-half_window:i+half_window]):
                valid_indices.append(chnk_cohPFvl.index[i])

        chnk_cohPFvl = chnk_cohPFvl.loc[valid_indices]

        # Calcular el promedio y desviación estándar de los registros anteriores y posteriores
        chnk_cohPFvl['mean_PF'] = chnk_cohPFvl['Valor'].rolling(window=window_size, center=True).mean()
        chnk_cohPFvl['std_PF'] = chnk_cohPFvl['Valor'].rolling(window=window_size, center=True).std()

        # Calcular los límites superior e inferior
        chnk_cohPFvl['lim_inf'] = chnk_cohPFvl['mean_PF'] - (3 * chnk_cohPFvl['std_PF'])
        chnk_cohPFvl['lim_sup'] = chnk_cohPFvl['mean_PF'] + (3 * chnk_cohPFvl['std_PF'])

        # Añadir mensajes de depuración para verificar los límites
        print(chnk_cohPFvl[['Fecha', 'Valor', 'mean_PF', 'std_PF', 'lim_inf', 'lim_sup']].head(20))

        # Máscara para identificar valores fuera de los límites
        mask_var5prev = (chnk_cohPFvl['Valor'] < chnk_cohPFvl['lim_inf']) | (chnk_cohPFvl['Valor'] > chnk_cohPFvl['lim_sup'])
        
        # Añadir mensajes de depuración para verificar los valores fuera de los límites
        print(chnk_cohPFvl[mask_var5prev][['Fecha', 'Valor', 'lim_inf', 'lim_sup']])
        
        # Lógica de etiquetado para 'Estado', sospechoso, '0PSO0'
        condicion_0PSO0 = mask_var5prev & (chnk_cohPFvl['Estado'].isnull())
        chnk_cohPFvl.loc[condicion_0PSO0, 'Estado'] = '0PSO0'
        mask_var5prev = mask_var5prev & ~condicion_0PSO0
        # 0PSO1
        condicion_0PSO1 = mask_var5prev & (chnk_cohPFvl['Estado'] == '0PSO0')
        chnk_cohPFvl.loc[condicion_0PSO1, 'Estado'] = '0PSO1'
        mask_var5prev = mask_var5prev & ~condicion_0PSO1
        # 0PSO2
        condicion_0PSO2 = mask_var5prev & (chnk_cohPFvl['Estado'] == '0PSO1')
        chnk_cohPFvl.loc[condicion_0PSO2, 'Estado'] = '0PSO2'
    
        # Se asegura la verificación de los valores anteriores si hubo cambio de chunk
        self.last_rows = chnk_cohPFvl.tail(5)

        # Eliminar las columnas temporales antes de devolver el chunk
        if 'Fecha_anterior' in chunk.columns:
            chunk.drop(columns=['Fecha_anterior','Delta_tiempo'], axis=1, inplace=True)
            
        # Copiar datos de chunk_coher al chunk original
        chunk.loc[chnk_cohPFvl.index] = chnk_cohPFvl
        # Continuar eliminando filas
        if 'mean_5' in chunk.columns:
            chunk.drop(columns=['mean_5','std_5','lim_inf','lim_sup'], axis=1, inplace=True)
        
        return chunk, mask_var5prev

    def p_varminh(self, chunk, archivo):
        '''Esta prueba detecta los valores horarios que no varían en 1.0 % durante la hora'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = None
        # Se genera la columna 'Estado_Anterior' si no existe
        if 'Estado_Anterior' not in chunk.columns:
            chunk['Estado_Anterior'] = None
    
        # Se toma nuevamente el archivo de frecuencias para analizar datos estrictamente consecutivos
        freqinst200b = pd.read_csv('EMAHR_Allinfo_Replcbl.csv', encoding='latin-1')
    
        # Obtener la frecuencia de 'freqinst200b' basado en 'Station' y asignar a 'periodos'
        sttn_code = chunk['Station'].values[0]
        if pd.isna(sttn_code):
            periodos = None
        else:
            freqinst200b_station = freqinst200b.loc[freqinst200b['Station'] == sttn_code]
            if freqinst200b_station.empty:
                print(f"No se encontró la estación {sttn_code} en freqinst200b")
                return chunk, None
            # Se toma el valor inferido de la frecuencia
            freq_inf_value = freqinst200b_station['FreqInf'].values[0]
            # Si el valor inferido es un NaN, se infiere dentro del código
            if pd.isna(freq_inf_value):
                try:
                    periodos = pd.infer_freq(chunk['Fecha'][-25:])
                    if periodos is None:
                        print(f"Frecuencia inferida es None para el archivo {archivo}")
                        return chunk, None
                except ValueError as e:
                    print(f'Error al inferir la frecuencia en el archivo {archivo}: {str(e)}')
                    return chunk, None
            else:
                periodos = freq_inf_value
    
        if periodos is None:
            print(f"Periodo es None para el archivo {archivo}")
            return chunk, None
    
        # Asegurarse de que 'periodos' tenga un número antes de la unidad
        if periodos.isalpha():
            periodos = '1' + periodos
    
        # Ordenar el chunk por la columna 'Fecha'
        chunk = chunk.sort_values('Fecha').reset_index(drop=True)
    
        # Se genera filtro para no considerar datos ya catalogados como erróneos o atípicos
        if 'Estado' in chunk.columns and chunk['Estado'].notna().all():
            chunk_varhmn = chunk[~chunk['Estado'].str.startswith(('0PER', '0PAT'), na=False)].copy()
        else:
            chunk_varhmn = chunk.copy()
    
        # Crear una columna de diferencia temporal y otras columnas temporales
        chunk_varhmn['Fecha_anterior'] = chunk_varhmn['Fecha'].shift(1)
        chunk_varhmn['Delta_tiempo'] = chunk_varhmn['Fecha'] - chunk_varhmn['Fecha_anterior']
    
        # Crear una máscara para identificar filas consecutivas según la frecuencia esperada
        freq_map = {'H': '1H', 'T': '1T', '5T': '5T', '10T': '10T'}
        expected_delta = pd.to_timedelta(freq_map.get(periodos, '1H'))
        mask_consecut = chunk_varhmn['Delta_tiempo'] == expected_delta
    
        # Calcular la diferencia absoluta entre los valores consecutivos
        chunk_varhmn['Delta'] = chunk_varhmn['Valor'].diff().abs()
        chunk_varhmn['Delta'] = chunk_varhmn['Delta'].where(mask_consecut)
    
        # Aplicar la máscara de las horas soleadas después de crear las columnas temporales
        mask_sunny2 = (chunk_varhmn['Fecha'].dt.hour >= 6) & (chunk_varhmn['Fecha'].dt.hour <= 18)
        mask_sun2 = chunk_varhmn[mask_sunny2]
    
        if periodos == 'H':
            # Máscara para identificar variaciones menores a 1.0
            mask_varhmin = mask_sun2['Delta'] < 1.0
        else:
            # Agrupar por horas y verificar si alguna variación dentro de la hora excede 1.0
            mask_sun2['Fecha_hora'] = mask_sun2['Fecha'].dt.floor('H')
            hora_groups = mask_sun2.groupby('Fecha_hora')
            mask_varhmin = hora_groups['Delta'].transform(lambda x: (x < 1.0).any())
            
        # Lógica de etiquetado para 'Estado', sospechoso, '0PSO0'
        condicion_0PSO0 = mask_varhmin & (mask_sun2['Estado'].isnull())
        mask_sun2.loc[condicion_0PSO0, 'Estado'] = '0PSO0'
        mask_varhmin = mask_varhmin & ~condicion_0PSO0
        # 0PSO1
        condicion_0PSO1 = mask_varhmin & (mask_sun2['Estado'] == '0PSO0')
        mask_sun2.loc[condicion_0PSO1, 'Estado'] = '0PSO1'
        mask_varhmin = mask_varhmin & ~condicion_0PSO1
        # 0PSO2
        condicion_0PSO2 = mask_varhmin & (mask_sun2['Estado'] == '0PSO1')
        mask_sun2.loc[condicion_0PSO2, 'Estado'] = '0PSO2'
        mask_varhmin = mask_varhmin & ~condicion_0PSO2
        # 0PSO2
        condicion_0PSO3 = mask_varhmin & (mask_sun2['Estado'] == '0PSO2')
        mask_sun2.loc[condicion_0PSO3, 'Estado'] = '0PSO3'
    
        # Copiar datos de chunk_jmp al chunk original
        chunk.loc[mask_sun2.index] = mask_sun2
    
        # Eliminar las columnas temporales antes de devolver el chunk
        columns_to_drop = ['Delta', 'Fecha_anterior', 'Delta_tiempo', 'Fecha_hora']
        columns_to_drop = [col for col in columns_to_drop if col in chunk.columns]
        chunk.drop(columns=columns_to_drop, axis=1, inplace=True)
    
        return chunk, mask_varhmin

    def procesar_archivos(self, funcion_evaluacion):
        '''Este método procesa la lectura y guardado de los archivos para todas las pruebas'''
        archivos = self.ruta_archivos

        archivos_salida = []  # Lista para almacenar nombres de archivos de salida

        # Se recorre cada archivo en la carpeta
        for archivo in archivos:
            if archivo.endswith('.csv'):
                ruta_archivo = os.path.join(self.dir_files, archivo)

                reader = pd.read_csv(ruta_archivo, encoding='latin-1', chunksize=self.chunk_size)#,dtype={7: 'str'}, low_memory=False)
                resultados = []

                for chunk in reader:
                    try:
                        chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
                    except ValueError:
                        chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S')

                    chunk['Station'] = chunk['Station'].astype('int64')
                    try:
                        chunk_resultado, _ = funcion_evaluacion(chunk, archivo)  # Desempaqueta solo el DataFrame
                    except ValueError:
                        chunk_resultado = funcion_evaluacion(chunk, archivo)  # Desempaqueta solo el DataFrame
                    resultados.append(chunk_resultado)

                if not resultados:  # Se verifica si la lista está vacía
                    self.logger.warning('No hay resultados válidos para concatenar en el archivo %s. Continuando con el siguiente archivo.', archivo)
                    continue
                    
                resultados_consolidados = pd.concat(resultados)

                # Genera el nombre del archivo de salida conservando los primeros 19 caracteres del nombre del archivo original
                nombre_archivo_salida = archivo[:19] + '_qc.csv'

                resultados_consolidados.to_csv(os.path.join(self.dir_files, nombre_archivo_salida), encoding='latin-1', index=False)

                archivos_salida.append(nombre_archivo_salida)  # Agregar el nombre del archivo a la lista
            
        # Actualiza self.ruta_archivos para que la próxima prueba procese los resultados de esta prueba
        self.ruta_archivos = archivos_salida
        # Se fija el log de procesamiento completo de archivos
        self.logger.info('Procesamiento completo de archivos de estaciones HR. Archivos generados: %s', archivos_salida)

In [5]:
procesador = AutomatHREMA('Test_QC')

In [6]:
procesador.procesar_archivos(procesador.p_transm)

C:\Users\user\AppData\Local\Temp\ipykernel_9636\71176568.py:71: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0PSO0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  chunk.loc[chunk['Fecha'].dt.floor('H') == row['Fecha'].floor('H'), 'Estado'] = '0PSO0'
C:\Users\user\AppData\Local\Temp\ipykernel_9636\71176568.py:71: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0PSO0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  chunk.loc[chunk['Fecha'].dt.floor('H') == row['Fecha'].floor('H'), 'Estado'] = '0PSO0'
C:\Users\user\AppData\Local\Temp\ipykernel_9636\71176568.py:71: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0PSO0' has dtype incompatible with float64, please explicitly cast to a compatible d

In [7]:
procesador.procesar_archivos(procesador.p_estruct)

In [8]:
procesador.procesar_archivos(procesador.p_limrig)

C:\Users\user\AppData\Local\Temp\ipykernel_9636\71176568.py:174: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  chunk.loc[condicion_0PSO0, 'Estado_Anterior'] = chunk.loc[condicion_0PSO0, 'Estado']
C:\Users\user\AppData\Local\Temp\ipykernel_9636\71176568.py:174: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  chunk.loc[condicion_0PSO0, 'Estado_Anterior'] = chunk.loc[condicion_0PSO0, 'Estado']
C:\Users\user\AppData\Local\Temp\ipykernel_9636\71176568.py:174: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first

In [21]:
procesador.procesar_archivos(procesador.p_persist)

C:\Users\user\AppData\Local\Temp\ipykernel_8248\2349811916.py:224: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0'
 '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0'
 '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0'
 '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0'
 '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0'
 '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0'
 '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0'
 '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0'
 '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0'
 '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0'
 '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0' '0PSO0'
 '0PSO0' '0P

IndexError: boolean index did not match indexed array along dimension 0; dimension is 54004 but corresponding boolean dimension is 54028

In [ ]:
procesador.procesar_archivos(procesador.p_salto)

In [ ]:
procesador.procesar_archivos(procesador.p_horavmaxmin)

In [ ]:
procesador.procesar_archivos(procesador.p_sigma)

In [9]:
procesador.procesar_archivos(procesador.p_coherPFvals)

                 Fecha  Valor  mean_PF  std_PF  lim_inf  lim_sup
37 2016-06-02 17:40:00   90.0      NaN     NaN      NaN      NaN
38 2016-06-02 17:45:00   90.0      NaN     NaN      NaN      NaN
39 2016-06-02 17:50:00   89.0      NaN     NaN      NaN      NaN
40 2016-06-02 17:55:00   89.0      NaN     NaN      NaN      NaN
41 2016-06-02 18:00:00   89.0      NaN     NaN      NaN      NaN
42 2016-06-02 18:05:00   90.0      NaN     NaN      NaN      NaN
43 2016-06-02 18:10:00   90.0      NaN     NaN      NaN      NaN
44 2016-06-02 18:15:00   91.0      NaN     NaN      NaN      NaN
45 2016-06-02 18:20:00   91.0      NaN     NaN      NaN      NaN
46 2016-06-02 18:25:00   92.0      NaN     NaN      NaN      NaN
47 2016-06-02 18:30:00   92.0      NaN     NaN      NaN      NaN
48 2016-06-02 18:35:00   92.0      NaN     NaN      NaN      NaN
49 2016-06-02 18:40:00   93.0      NaN     NaN      NaN      NaN
50 2016-06-02 18:45:00   93.0      NaN     NaN      NaN      NaN
51 2016-06-02 18:50:00   

In [ ]:
procesador.procesar_archivos(procesador.p_varminh)

In [24]:
import pandas as pd
import numpy as np

# Crear datos de prueba
def crear_datos_prueba(frecuencia, num_datos):
    fechas = pd.date_range(start='2024-01-01', periods=num_datos, freq=frecuencia)
    valores = np.random.normal(loc=100, scale=10, size=num_datos)
    datos = {'Fecha': fechas, 'Valor': valores, 'Station': ['TestStation'] * num_datos}
    df_prueba = pd.DataFrame(datos)
    return df_prueba

# Simular la clase y la función
class Validador:
    def __init__(self):
        self.last_rows = None
        self.current_file = None
    
    def p_coher5vals(self, chunk, archivo):
        '''Esta prueba verifica que un valor tenga coherencia con los 5 anteriores y 5 posteriores según su desviación estándar y media'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = None    
        # Se genera la columna 'Estado_anterior' si no existe
        if 'Estado_Anterior' not in chunk.columns:
            chunk['Estado_Anterior'] = None

        ## Trabajo con frecuencias
        # Asumimos un periodo fijo para simplificar
        periodos = '5T'
    
        if periodos is None:
            print(f"Periodo es None para el archivo {archivo}")
            return chunk, None

        ## Se asegura la revisión de 5 datos anteriores aún si cambia el chunk
        # Verificar si el archivo ha cambiado
        if self.current_file != archivo:
            # Si el archivo cambió, resetea self.last_rows y actualiza self.current_file
            self.last_rows = None
            self.current_file = archivo
        # Usar self.last_rows para concatenar con el chunk actual
        if self.last_rows is not None:
            chunk = pd.concat([self.last_rows, chunk])
            chunk.reset_index(drop=True)

        # Ordenar el chunk por la columna 'Fecha'
        chunk = chunk.sort_values('Fecha').reset_index(drop=True)

        # Asegurarse de que 'periodos' tenga un número antes de la unidad
        if periodos.isalpha():
            periodos = '1' + periodos

        # Crear una columna de diferencia temporal
        chunk['Fecha_anterior'] = chunk['Fecha'].shift(1)
        chunk['Delta_tiempo'] = chunk['Fecha'] - chunk['Fecha_anterior']

        # Se genera filtro para no considerar datos ya catalogados como erróneos
        if 'Estado' not in chunk.columns or chunk['Estado'].isnull().all():
            chunk_5vals = chunk.copy()
        else:
            chunk_5vals = chunk[~chunk['Estado'].str.startswith(('0PER','0PAT'), na=False)].copy()

        # Crear una máscara para identificar filas consecutivas según la frecuencia esperada
        mask_consecutivo = chunk_5vals['Delta_tiempo'] == pd.to_timedelta(periodos)
        chunk_5vals['consec_group'] = (~mask_consecutivo).cumsum()

        # Se establecen diferentes ventanas según frecuencias
        windows = {'T': {'window': 240}, '5T': {'window': 72}, '10T': {'window': 48}, 'H': {'window': 11}}
        window_size = windows[periodos]['window']
        half_window = window_size // 2

        # Filtrar grupos que tienen al menos el tamaño de ventana necesario
        group_counts = chunk_5vals['consec_group'].value_counts()
        valid_groups = group_counts[group_counts >= window_size].index
        chnk_coh5vl = chunk_5vals[chunk_5vals['consec_group'].isin(valid_groups)]

        # Verificar que los datos anteriores y posteriores sean consecutivos
        valid_indices = []
        for i in range(half_window, len(chnk_coh5vl) - half_window):
            if all(mask_consecutivo[i-half_window:i+half_window]):
                valid_indices.append(chnk_coh5vl.index[i])

        chnk_coh5vl = chnk_coh5vl.loc[valid_indices]

        # Calcular el promedio y desviación estándar de los registros anteriores y posteriores
        chnk_coh5vl['mean_5'] = chnk_coh5vl['Valor'].rolling(window=window_size, center=True).mean()
        chnk_coh5vl['std_5'] = chnk_coh5vl['Valor'].rolling(window=window_size, center=True).std()

        # Calcular los límites superior e inferior
        chnk_coh5vl['lim_inf'] = chnk_coh5vl['mean_5'] - (3 * chnk_coh5vl['std_5'])
        chnk_coh5vl['lim_sup'] = chnk_coh5vl['mean_5'] + (3 * chnk_coh5vl['std_5'])

        # Añadir mensajes de depuración para verificar los límites
        print(chnk_coh5vl[['Fecha', 'Valor', 'mean_5', 'std_5', 'lim_inf', 'lim_sup']].head(20))

        # Máscara para identificar valores fuera de los límites
        mask_var5prev = (chnk_coh5vl['Valor'] < chnk_coh5vl['lim_inf']) | (chnk_coh5vl['Valor'] > chnk_coh5vl['lim_sup'])
        
        # Añadir mensajes de depuración para verificar los valores fuera de los límites
        print(chnk_coh5vl[mask_var5prev][['Fecha', 'Valor', 'lim_inf', 'lim_sup']])

        # Lógica de etiquetado para 'Estado', sospechoso, '0PSO0'
        condicion_0PSO0 = mask_var5prev & (chnk_coh5vl['Estado'].isnull())
        chnk_coh5vl.loc[condicion_0PSO0, 'Estado'] = '0PSO0'
        mask_var5prev = mask_var5prev & ~condicion_0PSO0
        # 0PSO1
        condicion_0PSO1 = mask_var5prev & (chnk_coh5vl['Estado'] == '0PSO0')
        chnk_coh5vl.loc[condicion_0PSO1, 'Estado'] = '0PSO1'
        mask_var5prev = mask_var5prev & ~condicion_0PSO1
        # 0PSO2
        condicion_0PSO2 = mask_var5prev & (chnk_coh5vl['Estado'] == '0PSO1')
        chnk_coh5vl.loc[condicion_0PSO2, 'Estado'] = '0PSO2'

        # Se asegura la verificación de los valores anteriores si hubo cambio de chunk
        self.last_rows = chnk_coh5vl.tail(5)

        # Eliminar las columnas temporales antes de devolver el chunk
        if 'Fecha_anterior' in chunk.columns:
            chunk.drop(columns=['Fecha_anterior','Delta_tiempo'], axis=1, inplace=True)

        # Copiar datos de chunk_coher al chunk original
        chunk.loc[chnk_coh5vl.index] = chnk_coh5vl
        # Continuar eliminando filas
        if 'mean_5' in chunk.columns:
            chunk.drop(columns=['mean_5','std_5','lim_inf','lim_sup'], axis=1, inplace=True)

        return chunk, mask_var5prev

# Probar la función
def probar_p_coher5vals():
    # Crear un conjunto de datos de prueba
    df_prueba = crear_datos_prueba('5T', 8000)  # Cambia '5T' y el número de datos según sea necesario

    # Crear instancia de la clase Validador
    validador = Validador()
    
    # Introducir algunos valores fuera de los límites para probar el etiquetado
    df_prueba.loc[10:15, 'Valor'] = 200  # Valores anómalos que deberían ser etiquetados

    # Aplicar la función
    resultado_chunk, mask_var5prev = validador.p_coher5vals(df_prueba, 'archivo_prueba.csv')

    # Guardar el resultado en un CSV para revisión manual
    resultado_chunk.to_csv('resultado_prueba.csv', index=False)
    
    # Verificar si el resultado parece correcto
    print("Revisar el archivo 'resultado_prueba.csv' para verificar los resultados.")

# Ejecutar la prueba
probar_p_coher5vals()

                 Fecha       Valor  mean_5  std_5  lim_inf  lim_sup
37 2024-01-01 03:05:00  104.640878     NaN    NaN      NaN      NaN
38 2024-01-01 03:10:00   99.756331     NaN    NaN      NaN      NaN
39 2024-01-01 03:15:00   85.710635     NaN    NaN      NaN      NaN
40 2024-01-01 03:20:00  107.027710     NaN    NaN      NaN      NaN
41 2024-01-01 03:25:00   87.466289     NaN    NaN      NaN      NaN
42 2024-01-01 03:30:00   83.890767     NaN    NaN      NaN      NaN
43 2024-01-01 03:35:00  103.235479     NaN    NaN      NaN      NaN
44 2024-01-01 03:40:00  103.962081     NaN    NaN      NaN      NaN
45 2024-01-01 03:45:00   99.870471     NaN    NaN      NaN      NaN
46 2024-01-01 03:50:00   90.447196     NaN    NaN      NaN      NaN
47 2024-01-01 03:55:00   97.913549     NaN    NaN      NaN      NaN
48 2024-01-01 04:00:00   84.935664     NaN    NaN      NaN      NaN
49 2024-01-01 04:05:00   88.966224     NaN    NaN      NaN      NaN
50 2024-01-01 04:10:00  100.865845     NaN    Na